In [2]:
!pip install openfermion==0.11.0
!pip install openfermionpyscf
!pip install dwave-ocean-sdk

     |████████████████████████████████| 631 kB 21.6 MB/s 
  Created wheel for openfermion: filename=openfermion-0.11.0-py3-none-any.whl size=741959 sha256=5bc983ac7db0aabbb067d3b712087c9995a73e5d9e34d10f0b5b2af90f5b3326
  Stored in directory: /root/.cache/pip/wheels/60/71/10/0576f24aa73ae3e655fcb28a69d33a277959a5b0255d3a627b
  Created wheel for pubchempy: filename=PubChemPy-1.0.4-py3-none-any.whl size=13835 sha256=f93fb1bdcde5ecc0e72f7d1064b0de79ba21ab54f5580454db2cbde58c5540e3
  Stored in directory: /root/.cache/pip/wheels/7c/3d/8c/8192697412e9899dc55bbbb08bbc1197bef333caaa2a71c448
Successfully built openfermion pubchempy
     |████████████████████████████████| 13.1 MB 64 kB/s 
     |████████████████████████████████| 887 kB 36.3 MB/s 
     |████████████████████████████████| 88 kB 7.7 MB/s 
     |████████████████████████████████| 697 kB 43.2 MB/s 
     |████████████████████████████████| 106 kB 63.5 MB/s 
     |████████████████████████████████| 397 kB 61.1 MB/s 
     |██████████████████

In [10]:
import pyscf
from qiskit_nature.drivers import Molecule
from qiskit_nature.drivers.second_quantization import ElectronicStructureDriverType, ElectronicStructureMoleculeDriver
from qiskit_nature.drivers.second_quantization.pyscfd import PySCFDriver

In [11]:
import math

# from openfermionpyscf import run_pyscf
from openfermion.transforms import binary_code_transform, bravyi_kitaev_code, get_fermion_operator
from openfermion.hamiltonians import MolecularData
from openfermion.ops import FermionOperator, QubitOperator
from openfermion.utils import count_qubits
from pyscf import gto, scf, mcscf

from helper_functions import *
from XBK_method import *
from QCC_method import *

In [12]:
#create molecule
name = 'H3'
charge = 1
multiplicity = 1
basis = 'sto-6g'

bond_length = 1.1
geometry = get_molGeometry(name, bond_length)
    
molecule = MolecularData(
    geometry=geometry,
    basis=basis,
    multiplicity=multiplicity,
    charge=charge
)

In [19]:
#run RHF calculations
molecule = "H .0 .0 .0; H .0 .0 0.739"
driver = PySCFDriver(atom=molecule)
molecule = driver.run()
# hf_energy = float(molecule.hf_energy)
# hf_data = molecule._pyscf_data['scf']

# print(hf_energy)

In [4]:
#define active space
n_active_electrons = 2
n_active_orbitals = 3
occupied_indices, active_indices = get_active_space(molecule, n_active_electrons, n_active_orbitals)

#run CASCI calculations
casci_data = hf_data.CASCI(n_active_orbitals, n_active_electrons).run(verbose=False)
casci_energy = float(casci_data.e_tot)

print(casci_energy)

-1.2722647705358225


In [20]:
#convert to fermionic Hamiltonian
molecular_H = molecule.get_molecular_hamiltonian(occupied_indices=occupied_indices, active_indices=active_indices)
if molecular_H[()] == None:
    molecular_H[()] = 0
fermionic_H = get_fermion_operator(molecular_H)

#add penalty term to ensure correct number of electrons in ground state
weight = 5
penalty_term = FermionOperator('', n_active_electrons)

for i in range(molecular_H.n_qubits):
    penalty_term += FermionOperator(str(i)+'^ '+str(i), -1)
fermionic_H += weight*penalty_term**2

print(fermionic_H)

AttributeError: 'ElectronicStructureDriverResult' object has no attribute 'get_molecular_hamiltonian'

In [6]:
#convert to Pauli operator Hamiltonian
binary_code = bravyi_kitaev_code(molecular_H.n_qubits)
qubit_H = binary_code_transform(fermionic_H, binary_code)
qubit_H.compress()

#apply symmetry reductions and calculate minimum eigenvalue (should be equal to CASCI energy)
sectors = taper_qubits(qubit_H)
qubit_H, min_eigenvalue = sector_with_ground(sectors)
m = count_qubits(qubit_H)

print(min_eigenvalue, '\n')
print(qubit_H)

-1.2722647705358519 

12.052315964211402 [] +
-0.019297927866982666 [X0] +
0.019297927866982718 [X0 X1 X2] +
-0.015780732595205717 [X0 X1 Z2 X3] +
0.019297927866982666 [X0 X1 Z2 Z3] +
-0.015780732595205717 [X0 X1 X3] +
0.019297927866982666 [X0 X1 Z3] +
0.019297927866982718 [X0 Y1 Y2] +
0.015780732595205703 [X0 Z1 X2 X3] +
-0.01929792786698272 [X0 Z1 X2 Z3] +
0.01929792786698273 [X0 Z1 Z2] +
-0.015780732595205703 [X0 Z1 X3] +
0.01929792786698273 [X0 Z1 Z3] +
0.015780732595205703 [X0 X2 X3] +
-0.01929792786698272 [X0 X2 Z3] +
0.015780732595205738 [X0 Z2 X3] +
-0.019297927866982666 [X0 Z2 Z3] +
-0.015780732595205717 [Y0 X1 X2 Y3] +
-0.015780732595205717 [Y0 X1 Y2 X3] +
0.019297927866982718 [Y0 X1 Y2 Z3] +
0.015780732595205717 [Y0 Y1 X2 X3] +
-0.019297927866982718 [Y0 Y1 X2 Z3] +
-0.015780732595205717 [Y0 Y1 Y2 Y3] +
-0.015780732595205717 [Y0 Y1 Z2 X3] +
0.01929792786698273 [Y0 Y1 Z2 Z3] +
-0.015780732595205717 [Y0 Y1 X3] +
0.01929792786698273 [Y0 Y1 Z3] +
0.015780732595205738 [Y0 Z1 Y2 X3

In [7]:
#set sampler to perform the annealing
from neal import SimulatedAnnealingSampler
sampler = SimulatedAnnealingSampler() #uses simulated annealing, see D-Wave's ocean sdk for more options

In [15]:
### XBK method ###

#set r value
r = 4

#construct qubit Hamiltonians and C terms for XBK method
qubit_Hs, qubit_Cs = [],[]
for p in range(int(math.ceil(r/2+1))):
    qubit_Hs += [XBK_transform(qubit_H, r, p)]
    qubit_Cs += [construct_C(m, r, p)]

#run XBK method
XBK_energy, ground_state = XBK(qubit_Hs, qubit_Cs, r, sampler, starting_lam=0, num_samples=1000, strength=1e3, verbose=False)

print(XBK_energy)
print(ground_state) #ground state in rm-qubit space

-1.24006930460746
[1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1]


In [16]:
### QCC method ###

#set number of Bloch angle and entangler amplitude foldings
angle_folds = 2
amplitude_folds = 1

#create dictionary of QubitOperator entanglers
entanglers = {'IYZI': QubitOperator('Y1 Z2'), 'IZYI': QubitOperator('Z1 Y2'),
              'IXYI': QubitOperator('X1 Y2'), 'IYXI': QubitOperator('Y1 X2')}

#run QCC method
QCC_energy, variables = QCC(qubit_H, entanglers, angle_folds, amplitude_folds, sampler,
                            num_cycles=10, num_samples=1000, strength=1e3)
    
print(QCC_energy)
print(variables)

-1.2664583599980688
{phi0: 1.3564044038755205, phi1: 6.283185307179586, phi2: 3.5834420620660397, phi3: 3.1465829731980053, the0: 4.554931150993846e-06, the1: 3.141592653589793, the2: 2.827524175524541e-07, the3: 0.0, tau0: 2.9829102234933917, tau1: 6.12449846615571, tau2: 6.283185307179586, tau3: 0.0}
